# CART Decision Tree for Optical Character Recognition

Here, we build a decision tree to classify between two handwritten numbers, 3 and 5. The data set used is from Oregon State University, Fall 2018 CS 534 Machine Learning Course Implementation Assignment 3.<br><br>
There are 100 features for each sample, corresponding to the top 100 principal components generated using PCA from the original sample features of size 28 * 28.<br><br>
The training set contains 4888 samples. Each sample is a list of 101 values. The first column represents the digit’s label  which is 3 or 5. The other 100 floating values are the PCA-generated features for this sample.<br><br>
The validation set contains 1629 samples. This set will be used to tune the hyper parameters and select the best model.<br>
No test set is provided.<br>

# Import Libraries

In [ ]:
import numpy as np

## Preprocessing
Decision Tree algorithm does not need any scaling of the features.

In [1]:
class Preprocessing:
    
    def __init__(self, raw_training_data_path, raw_validation_data_path):
            
            # Read the training and validation data. As the features are just PCA components, no need to observe the 
            # column names, so can proceed with numpy instead of pandas.
            
            self.raw_training_data = np.genfromtxt(raw_training_data_path, delimiter = ",")
            self.raw_validation_data = np.genfromtxt(raw_validation_data_path, delimiter = ",")
            
            
            
            # split training features and labels
            self.training_features = self.raw_training_data[:, 0:-1]
            self.training_labels = self.raw_training_data[:, -1]
            
            print("No of training samples: {}, No of training features: {}".format(len(self.training_labels), self.training_features.shape[1]))
            
            
            # split validation features and labels
            self.validation_features = self.raw_validation_data[:, 0:-1]
            self.validation_labels = self.raw_validation_data[:, -1]
            
            print("No of validation samples: {}".format(len(self.training_labels), ))

        
        

# Decision Tree Implementation with GINI Impurity

In [ ]:
class Decision_Tree_Node:
    
    def __init__(self, X, y, depth):
        self.X = X
        self.y = y
        self.left_X = None
        self.right_X = None
        self.left_Y = None
        self.right_Y = None
        self.depth = depth
        self.feature_index_with_optimal_split = None
        self.threshold_for_optimal_split = None # since features are real values
        self.is_leaf = False


class Deicision_Tree(BaseEstimator, ClassifierMixin):
    
    '''
    Implementation of CART Decision Tree.
    
    Parameters
    -------------
    max_depth : int, optional, default 100
                The maximum depth of the deicison tree for early-stopping/ to avoid overfitting to outliers.
    min_pool :  int, optional, default 10
                The minimum number of samples at a node after split for early-stopping/ to avoid overfitting.
    '''
    
    
    
    def __init__(self, max_depth = 100, min_pool = 10,):
        
        self.root = Decision_Tree_Node(0)
        self.max_depth = 100
        self.min_samples = 10
        self.depth = 0
        
        
    
    
    def build_decision_tree(self, X, y, node, current_depth):
        '''
        Recursively builds the decision tree if the terminal conditions not met
        
        Parameters
        -----------------
        X : training features
        y : training labels
        '''
        
        
        # base cases of recursion
        if current_depth == self.max_depth:
            node.is_leaf = True
            return
            
        elif len(training_features) <= self.min_pool:
            node.is_leaf = True
            return
            
        elif len(y) < self.min_pool:
            node.is_leaf = True
            return
        else:
            # find the best split and the feature that gives this best split
            feature_for_best_split, best_split_threshold, X_left, X_right, y_left, y_right = self.best_split(node.X, node.y, depth) 
            
            
            if feature_for_best_split is None:
                node.is_leaf = True
            else:
                
                # Recursively build trees to the left and the right using the best features and the best split threshold
                self.depth = current_depth
                self.feature_index_with_optimal_split = feature_for_best_split
                self.threshold_for_optimal_split = best_split_threshold
                
                
                left_node = Decision_Tree_Node(X_left, y_left, current_depth+1)
                rightt_node = Decision_Tree_Node(X_right, y_right, current_depth+1)
                
                
                self.build_decision_tree(X_left, y_left, left_node, current_depth+1)
                self.build_decision_tree(X_right, y_right, right_node, current_depth+1)
            
        
    
        
        
        
        
        
    def best_split(self, X, y):
        '''
        Divides the training samples into optimal splits according to the impurity metric
        '''
        
        
        
        
        
        gini_impurity_before_split = self.calculate_gini_impurity(y)
        lowest_impurity_till_now = gini_impurity_before_split
        feature_for_best_split = None
        threshold_for_best_split = None
        
        # for each feature, find the GINI gain at each threshold
        for feature in range(X.shape[1]):
            
            # make a new copy of the data set features and labels
            X_copy = copy.deepcopy(X)
            y_copy = copy.deepcopy(y)
            
            current_feature = X_copy[:, feature]
            
            # TRICK: Sort the training samples according to this feature's values
            # and find GINI only at samples where the class label changes.
            
            # sort according to the current feature
            sorting_indices = X_copy[:,feature].argsort()
            X_copy_sorted = X_copy[sorting_indices]
            
            y_copy_sorted = y_copy[sorting_indices]
            
            thresholds = []
            threshold_impurities = []
            best_threhold = None
            
            
            prev_label = None
            
            for sample_num in range(len(X_copy)):
                
                if prev_label is None:
                    prev_label = y_copy_sorted[sample_num]
                
                if prev_label is not None: # this means that this is not the first sample
                    
                    # if the class label changed, set this feature value as the threshold
                    # make sure you do not split if the sample number is less
                    if y_copy_sorted[sample_num] != prev_label and sample_num >= self.min_samples and len(y)-sample_num >= self.min_samples:
                        
                        threshold = X_copy_sorted[sample_num,feature]
                        thresholds.append(threshold)
                        
                        
                        # all the indices below sample num belong to the left child
                        
                        # calculate gini impurities
                        below_threshold_impurity = self.calculate_impurity(y_copy_sorted[0:sample_num])
                        above_threshold_impurity = self.calculate_impurity(y_copy_sorted[sample_num:])
                        
                        
                        # calculate gain
                        prob1 = sample_num+1/len(y)
                        prob2 = (len(y) - (sample_num+1) )/len(y)
                        
                        threhold_impurity = (prob1 * below_threshold_impurity + prob2 * above_threshold_impurity)
                        threshold_impurities.append(threshold_impurity)
                        
                        
                        if threhold_impurity < lowest_impurity_till_now:
                            lowest_impurity_till_now = threhold_impurity
                            feature_for_best_split = feature
                            threshold_for_best_split = threshold
                            
                            
                        
                        # update prev_label
                        prev_label = y_copy_sorted[sample_num]
                        
                
                    
        # if no threshold gave a split thats better than a split previously found in another feature's split
        if lowest_impurity == gini_impurity_before_split:
            return None
        else:
            
            # make the best split and return
            
            left_indices = X[:, best_feature] < best_split_threshold
            right_indices = X[:, best_feature] >= best_split_threshold
            
            x_left = X[left_indices]
            x_right = X[right_indices]
            
            y_left = y[left_indices]
            y_right = y[right_indices]
            
            
            
            
            return feature_for_best_split, best_split_threshold, X_left, X_right, y_left, y_right
                
                        
                
                
            
                
                
            
            
        
        
        
    
    def split(self, X, y):
        
    
    
    
    
    def calculate_gini_impurity(self, labels):
        '''
        Calculate GINI impurity metric U at a node of the decision tree
        
        U = 1 - [(p^2) + (q^2 )]
        
        where p = No of samples at this node with feature x below the node threshold/ No of samples at this node
              q = No of samples at this node with feature x equal or above the node threshold/ No of samples at this node
        
        Parameters
        -----------------
        
        
        '''
        probabilities = []
        
        for class_label in self.unique_labels:
            probability_of_class = labels[labels == class_label]/ len(labels)
            probabilities.append(probability_of_class)
        
        impurity = 1 + sum([elem^2 for elem in probabilities])
        
        assert impurity <= 1
        
        return impurity
        
            
            
    
    
    
        
        
        
        
        
    
    def fit(self, X, y):
        
        '''
        Train CART decision tree using GINI impurity.
        
        Parameters
        ----------------
        X : arraylike
            The training samples in 2D format, corresponding to input features of each sample.
        y : arraylike, optional, default None
            The labels for each sample in X. Some algorithms do not require the label y while some (like Decisoin Tree) do.
            Hence, keep y as optional.
        
        Returns
        ----------------
        self : Trained Decision Tree object
               This method returns self for compatibility reasons with sklearn's other interfaces/ functionalities.
        
        
        '''
        self.root_node = Decision_Tree_Node(X, y, 0)
        self.build_decision_tree(X, y, self.root_node, 0)
    
        
        
        